In [1]:
import cv2
import numpy as np
import sounddevice as sd
import mediapipe as mp
import threading
import time
from playsound import playsound

# === Load background images ===
IMG_SIZE = (640, 480)
img_idle = cv2.resize(cv2.imread('poto.jpg'), IMG_SIZE)
img_cheer = cv2.resize(cv2.imread('poto1.jpg'), IMG_SIZE)
img_clap = cv2.resize(cv2.imread('poto2.jpg'), IMG_SIZE)

# === Shared state ===
status_lock = threading.Lock()
current_status = "idle"  # idle, cheer, clap

# === Inisialisasi segmentasi MediaPipe ===
mp_selfie = mp.solutions.selfie_segmentation
segmentor = mp_selfie.SelfieSegmentation(model_selection=0)

# === Fungsi update status + mainkan suara (hanya sekali) ===
def set_status(new_status, sound_path):
    global current_status
    with status_lock:
        if current_status != new_status:
            current_status = new_status
            if sound_path:
                threading.Thread(target=playsound, args=(sound_path,), daemon=True).start()

# === Deteksi suara keras ===
def audio_callback(indata, frames, time_, status):
    volume = np.linalg.norm(indata) * 10
    if volume > 10:
        set_status("cheer", "audio.wav")
    else:
        set_status("idle", None)

def audio_thread():
    with sd.InputStream(callback=audio_callback):
        sd.sleep(100000)

# === Kamera utama: tampilkan hasil dan deteksi gestur ===
def display_loop():
    global current_status

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=2)

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    if not cap.isOpened():
        print("❌ Kamera tidak tersedia.")
        return

    cv2.namedWindow("Filter Stadion", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Filter Stadion", *IMG_SIZE)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, IMG_SIZE)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # === MediaPipe Hand detection ===
        results = hands.process(rgb)
        tangan_terdeteksi = 0
        jarak = 0.0
        gesture_detected = False

        if results.multi_hand_landmarks:
            tangan_terdeteksi = len(results.multi_hand_landmarks)

            for hand_landmarks in results.multi_hand_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)

            if tangan_terdeteksi == 2:
                h1 = results.multi_hand_landmarks[0].landmark[0]
                h2 = results.multi_hand_landmarks[1].landmark[0]
                jarak = np.sqrt((h1.x - h2.x)**2 + (h1.y - h2.y)**2)

                if jarak < 0.35:
                    gesture_detected = True
                    set_status("clap", "audio2.wav")
                else:
                    if current_status == "clap":
                        set_status("idle", None)
        else:
            if current_status == "clap":
                set_status("idle", None)

        # Tambahkan info teks ke layar
        cv2.putText(frame, f"Tangan terdeteksi: {tangan_terdeteksi}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        if tangan_terdeteksi == 2:
            cv2.putText(frame, f"Jarak tangan: {jarak:.2f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # === Tentukan background saat ini ===
        with status_lock:
            if current_status == "cheer":
                bg = img_cheer
            elif current_status == "clap":
                bg = img_clap
            else:
                bg = img_idle

        frame_out = apply_virtual_background(frame, bg)
        cv2.imshow("Filter Stadion", frame_out)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        time.sleep(0.01)

    cap.release()
    cv2.destroyAllWindows()

# === Fungsi overlay background virtual ===
def apply_virtual_background(frame, bg_image):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = segmentor.process(rgb)
    mask = result.segmentation_mask > 0.5
    bg_resized = cv2.resize(bg_image, (frame.shape[1], frame.shape[0]))
    return np.where(mask[..., None], frame, bg_resized)

# === Jalankan semua thread dan tampilan utama ===
if __name__ == "__main__":
    threading.Thread(target=audio_thread, daemon=True).start()
    display_loop()


KeyboardInterrupt: 